In [2]:
import pygsheets
import pandas as pd
from entsoe import EntsoePandasClient
import seaborn as sns
import numpy as np
from datetime import datetime, date, timedelta

In [3]:
# Connecting to ENTSO-E API 

client = EntsoePandasClient(api_key='4b1229d3-1c2c-4fbb-9e43-4d1692208b57')

In [33]:
# timestamps
today = datetime.today()
today_previoushour = today.replace(hour = today.hour -1  ,minute=59,second = 59, microsecond=00)
PreviousDay = today_previoushour.replace(day = today.day -1)
start = pd.Timestamp(PreviousDay, tz='Europe/Brussels')
end = pd.Timestamp(today_previoushour, tz='Europe/Brussels')

In [ ]:
# QUERY 1: Actual Generation

# Queries of each country

Countries = ['PT','ES', 'FR','DE','BE','CZ','IT','AT', 'NL']
Generation = []

for i in Countries: 
    Query = client.query_generation(i, start=start,end=end, psr_type=None) # Generation Query 
    Query['Country'] = i
    Generation.append(Query)

Generation = pd.concat(Generation)

Generation.rename(columns=''.join, inplace=True)
Generation['Date'] = Generation.index.astype(str)
Generation['Date'] = pd.to_datetime(Generation['Date'])
#Unpitoving for Tableau format
Generation_lastmonth_unpivoted = Generation.melt(id_vars = ['Date','Country'])
    

In [34]:
# QUERY 2: Prices

Countries = ['ES','PT','FR','DE_LU','BE','CZ','IT_NORD','AT','NL','SK','FI','CH','PL','GR','EE','HU','RO','LT','LV','NO_1','RS','SI']

Prices = []

for i in Countries: 
    Query = client.query_day_ahead_prices(i, start=start,end=end).to_frame() # Prices Query 
    Query['Country'] = i
    Prices.append(Query)

Prices = pd.concat(Prices)

#Adding Luxembourg

Price_Luxembourg = client.query_day_ahead_prices('DE_LU', start=start,end=end).to_frame()
Price_Luxembourg['Country']="LU"
Prices = pd.concat([Prices,Price_Luxembourg])

#Replacing wrong country codes 

Prices['Country'].replace(to_replace= 'DE_LU',value='DE', inplace= True)
Prices['Country'].replace(to_replace= 'IT_NORD',value='IT', inplace= True)
Prices['Country'].replace(to_replace= 'NO_1',value='NO', inplace= True)
Prices['Country'].replace(to_replace= 'GB',value='UK', inplace= True)

Prices['Date'] = Prices.index.astype(str).str[:-6]
Prices['Date'] = pd.to_datetime(Prices['Date'])
#Unpitoving for Tableau format
Prices_unpivoted = Prices.melt(id_vars = ['Country','Date'])



In [35]:
# google sheets authentication
creds = '/Users/brunoluis/Jupyter/Projects /ENTSO API/root-gist-358222-ff123e36322d.json'
api = pygsheets.authorize(service_file=creds)
wb = api.open('ENTSO-E')

# open the sheet by name
sheet = wb.worksheet_by_title('Price')
sheet.set_dataframe(Prices_unpivoted, (1,1))


In [ ]:
#sns.set(rc={'figure.figsize':(12,9)})
#sns.lineplot(data = Prices_unpivoted,x = 'Date', y = 'value', hue = 'Country')